In [39]:
from lib.geometry import *
from lib.busstop import *
from lib.yahoo_transit import *
from lib.mapbox import *
import folium
import geopandas
from shapely.ops import unary_union
from dotenv import load_dotenv
from defines import *
from lib.busroute import *
from lib.buscompany import *

load_dotenv()

True

In [40]:
KAIT_GEO = Geometry(type="", long=139.3399782, lat=35.4861002)

MAPBOX_ATTR = """© <a href="https://www.mapbox.com/about/maps/">Mapbox</a> ©
<a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>
<strong
  ><a href="https://labs.mapbox.com/contribute/" target="_blank"
    >Improve this map</a
  ></strong
>"""


In [41]:
# 250円圏内

_routes = create_bus_routes_from_stops(KAIT_SCHOOL_ZONE_BUS_STOPS)
routes = [_routes[key] for key in _routes.keys()]
kanachu = BusCompany(id=routes[0].stops[0].group, routes=routes, stops=KAIT_SCHOOL_ZONE_BUS_STOPS)

stops_filtered: list[BusStop] = []
max_fare = 250
max_transfer = 0
for stop in KAIT_SCHOOL_ZONE_BUS_STOPS:
    for target_stop in [FRONT_OF_KAIT_BUS_STOP, INSIDE_KAIT_BUS_STOP]:
        if stop.name == target_stop.name:
            continue
        ride_infos = kanachu.get_ride_infos(target_stop.name, stop.name)
        ride_infos_filtered = []
        for ride_info in ride_infos:
            if ride_info["fare"] <= max_fare and ride_info["transfer"] <= max_transfer:
                stops_filtered.append(stop)

stops_filtered = list(set(stops_filtered))

# 地図作成
f = folium.Figure(width=600, height=400)
# まずはバス停のピンを刺すだけ
folium_map = folium.Map(
    location=KAIT_GEO.folium_coordinate(),
    zoom_start=14,
    tiles="https://cyberjapandata.gsi.go.jp/xyz/std/{z}/{x}/{y}.png",
    attr=f"出典: 国土地理院ウェブサイト・地理院タイル・標準地図 {MAPBOX_ATTR}"
).add_to(f)

for stop in stops_filtered+[FRONT_OF_KAIT_BUS_STOP, INSIDE_KAIT_BUS_STOP]:
    if stop is FRONT_OF_KAIT_BUS_STOP or stop is INSIDE_KAIT_BUS_STOP:
        icon = folium.Icon(color="blue", prefix="fa", icon="school")
    else:
        icon = folium.Icon(color="blue", prefix="fa", icon="bus")

    folium.Marker(
        location=stop.geometry.folium_coordinate(),
        icon=icon
    ).add_to(folium_map)
folium_map